In [66]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from urllib.parse import urlparse, parse_qs
from dateutil import parser,tz

In [67]:
# data = {
#     "keywords": "anies",
#     "since_time": "2023-09-01",
#     "until_time": "2023-11-10"
# }

In [68]:
# # input_date_str = datetime.strftime(data['since_time'], "%Y-%m-%d")
# input_date_str=data['since_time']
# print(input_date_str)
# # headers = {
# #         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
# #     }
# # url = f"https://index.okezone.com/bydate/channel/2023/09/01/1"
# # response = requests.get(url, headers=headers)
# # soup = BeautifulSoup(response.text, 'html.parser')
# # response = requests.get(url, headers=headers)
# # soup = BeautifulSoup(response.text, 'html.parser')
# # articles = soup.find_all('h4', {"class": "f17"})
# # for article in articles:
# #     link = article.find('a')['href']
# #     print(link)

In [69]:
# headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
#     }
# url = f"https://index.okezone.com/bydate/channel/2023/09/01/1"
# response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, 'html.parser')
# # print(soup)
# cek = soup.find('div', {"class": "time r1 fl bg1 b1"})
# links=cek.find_all('a')
# all_link=[]
# for link in links:
#         if link and 'href' in link.attrs:
#             link_href = link['href']
#             all_link.append(link_href)
# link=all_link[-1]
# # print(link)
# parsed_url = urlparse(link)
# # print(parsed_url)
# number = parsed_url.path.split("/")[-2]
# # print(number)

In [70]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
url = f"https://index.okezone.com/bydate/index/2023/12/01/0/"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
body= soup.find('div',{"class":"news-content"})

articles = body.find_all('div', {"class": "content-hardnews pad-index"})
    
links = []
for article in articles:
    link = article.find('a')['href']
    links.append(link)
    
print(links)

['https://nasional.okezone.com/read/2023/12/01/337/2931151/stafsus-jokowi-ajak-pemuda-suarakan-hentikan-peperangan', 'https://sports.okezone.com/read/2023/12/01/43/2931147/seru-enam-tim-memperebutkan-gelar-juara-liga-1-esports-nasional-2023', 'https://news.okezone.com/read/2023/12/01/340/2931145/tingkatkan-kesadaran-publik-tentang-isu-air-kementerian-pupr-gelar-pameran-water-art-installation', 'https://index.okezone.com/read/2023/12/01/612/2931018/ini-arti-mimpi-gigi-copot-takut-di-bawah-alam-sadar-hingga-depresi', 'https://news.okezone.com/read/2023/12/01/340/2931134/hadiri-munas-pramuka-di-aceh-atikoh-ganjar-pranowo-dapat-sambutan-hangat', 'https://celebrity.okezone.com/read/2023/12/01/205/2931144/summerlane-sukses-meriahkan-music-zone-by-okezone-radio', 'https://economy.okezone.com/read/2023/12/01/11/2931143/kontribusi-ekonomi-berkelanjutan-pegadaian-raih-dua-penghargaan-top-bumn-awards-2023', 'https://health.okezone.com/read/2023/12/01/483/2931139/6-bahaya-kebanyakan-makan-protein-

In [71]:
def scrape_links(date, page_number):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://index.okezone.com/bydate/index/{date}/{page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    body= soup.find('div',{"class":"news-content"})

    articles = body.find_all('div', {"class": "content-hardnews pad-index"})
    
    links = []
    for article in articles:
        link = article.find('a')['href']
        links.append(link)
        
    print(f"Scraped {len(links)} links from page {page_number} url {url}")
    return links

In [72]:
def scrape_link_per_day(date, max_threads=5):
    page_number = 0
    page_links = []
    # Adjust the date format here
    current_date = datetime.strptime(date, "%Y-%m-%d")
    formatted_date = current_date.strftime("%Y/%m/%d")
    # print(formatted_date)
    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        futures = []

        while True:
            future = executor.submit(scrape_links, formatted_date, page_number)
            futures.append(future)
            page_number += 15

            # Break the loop if no more articles are found
            if not future.result():
                break

        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [73]:
link=scrape_link_per_day('2023-11-02')

Scraped 15 links from page 0 url https://index.okezone.com/bydate/index/2023/11/02/0
Scraped 15 links from page 15 url https://index.okezone.com/bydate/index/2023/11/02/15
Scraped 15 links from page 30 url https://index.okezone.com/bydate/index/2023/11/02/30
Scraped 15 links from page 45 url https://index.okezone.com/bydate/index/2023/11/02/45
Scraped 15 links from page 60 url https://index.okezone.com/bydate/index/2023/11/02/60
Scraped 15 links from page 75 url https://index.okezone.com/bydate/index/2023/11/02/75
Scraped 15 links from page 90 url https://index.okezone.com/bydate/index/2023/11/02/90
Scraped 15 links from page 105 url https://index.okezone.com/bydate/index/2023/11/02/105
Scraped 15 links from page 120 url https://index.okezone.com/bydate/index/2023/11/02/120
Scraped 15 links from page 135 url https://index.okezone.com/bydate/index/2023/11/02/135
Scraped 15 links from page 150 url https://index.okezone.com/bydate/index/2023/11/02/150
Scraped 15 links from page 165 url ht

In [74]:
print(len(link))

704


In [75]:
def scrape_url(url, max_retries=2):
    retries = 0
    while retries < max_retries:
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
            }
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                # Judul Berita
                title_elem = soup.find('div', {"class": "title"})
                if title_elem:
                    title_text = title_elem.text.strip()
                else:
                    title_text = "Title not found"

                # Tanggal berita
                date_elem = soup.find('div', {"class": "reporter"})
                if date_elem:
                    date_text = date_elem.text.strip()
                    # Define a custom timezone information for WIB
                    def wib_tz(offset):
                        return tz.tzoffset('WIB', offset)

                    # Parse the date string with the custom timezone information
                    date_object = parser.parse(date_text, tzinfos={'WIB': 7*60}, fuzzy=True)

                    # Format the datetime object to the desired format
                    formatted_date = date_object.strftime('%Y-%m-%d')

                else:
                    date_text = "Date not found"

                # Content Berita
                body_elem = soup.find('div', {"class": "read"})

                if body_elem:
                    content_text = body_elem.text
                    content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                    content_text = ' '.join(content_text.split())
                else:
                    content_text = "Content not found"

                return {
                    'title': title_text,
                    'date': formatted_date,
                    'content': content_text,
                    'link': url
                }
            elif response.status_code == 429:
                print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                time.sleep(5)
            else:
                print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching URL '{url}': {e}")
        except Exception as e:
            print(f"Error processing URL '{url}': {e}")
        retries += 1
        if retries < max_retries:
            print(f"Retrying {url} (Attempt {retries}/{max_retries})")
            time.sleep(5)  # You can adjust the delay as needed
    return None

In [76]:
cek = scrape_url('https://nasional.okezone.com/read/2023/12/01/337/2931151/stafsus-jokowi-ajak-pemuda-suarakan-hentikan-peperangan')

In [77]:
cek

{'title': 'Stafsus Jokowi Ajak Pemuda Suarakan Hentikan Peperangan',
 'date': '2023-01-02',
 'content': 'DUBAI - Staf Khusus Presiden Jokowi, Diaz Hendropriyono tegaskan bahwa Indonesia siap berperan meminimalkan dampak perubahan iklim melalui inovasi-inovasi para ecopreneurs yang tersebar di seluruh Indonesia. Menjadi bagian dari gelaran tahunan PBB yang memobilisasi gerakan aksi global untuk membatasi kenaikan suhu dunia akibat pemanasan global atau Conference of the Parties (COP) ke-28, Diaz menyampaikan Indonesia mampu memberi harapan bagi dunia karena memiliki banyak ecopreneur yang menciptakan inovasi-inovasi produk ramah lingkungan dengan tujuan mengurangi emisi gas rumah kaca.BACA JUGA:Seru, Enam Tim Memperebutkan Gelar Juara Liga 1 Esports Nasional 2023Konferensi Tingkat Tinggi yang dilaksanakan di Dubai, Uni Emirat Arab (UEA) mulai dari 30 November ini dihadiri oleh puluhan ribu delegasi dari 199 Pihak Konvensi UN Framework Convention on Climate Change (UNFCCC).Dalam salah sa